Let us have a look at the phosphorylates dataset.

In [1]:
import torch

import pandas as pd
import numpy as np

from pathlib import Path

/home/jovyan/.conda/envs/bioblp-env/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
DATA_DIR = Path("/home/jovyan/workbench-shared-folder/bioblp/")

In [3]:
phospho_path = DATA_DIR.joinpath("data/benchmarks/transductive/phosphorylation.tsv")

phospho_df = pd.read_csv(phospho_path, sep="\t", header=None)
phospho_df.columns = ["src", "edg", "tgt"]

In [4]:
phospho_df

,src,edg,tgt
0,P06241,phosphorylates,P37840
1,Q9UQM7,phosphorylates,Q9UQL6
2,P06493,phosphorylates,P60891
3,Q15759,phosphorylates,P00533
4,P31749,phosphorylates,P19634
...,...,...,...
24544,P35626,phosphorylates,P51681
24545,O14578,phosphorylates,Q9UI36
24546,P17252,phosphorylates,Q8TD43
24547,P49841,phosphorylates,Q8NHW3


In [5]:
unique_entities = set(list(phospho_df.src.unique()) + list(phospho_df.tgt.unique()))

### negative sampling

In [6]:
from bioblp.benchmarking.preprocess import prepare_dpi_samples

In [7]:
phospho_prepped_df = prepare_dpi_samples(phospho_df, num_negs_per_pos=10)

Reconstructing all label-based triples. This is expensive and rarely needed.


In [8]:
resulting_entities_sampled = set(list(phospho_prepped_df.src.unique()) + list(phospho_prepped_df.tgt.unique()))

In [9]:
len(resulting_entities_sampled - unique_entities)

0

In [10]:
phospho_prepped_df

,src,edg,tgt,label
0,P06241,phosphorylates,P37840,1
1,Q9UQM7,phosphorylates,Q9UQL6,1
2,P06493,phosphorylates,P60891,1
3,Q15759,phosphorylates,P00533,1
4,P31749,phosphorylates,P19634,1
...,...,...,...,...
192944,Q9Y6R4,phosphorylates,Q8N4C6,0
192945,Q9Y6R4,phosphorylates,Q92685,0
192946,P27448,phosphorylates,Q9Y6R4,0
192947,Q9Y6R4,phosphorylates,Q2M2Z5,0


### Featurise

In [12]:
print("getting protein embeddings")

prot_embs_path = DATA_DIR.joinpath("data/protein_data/protein_embeddings_full_24_12.pt")

embeddings = torch.load(prot_embs_path)


getting protein embeddings


RuntimeError: PytorchStreamReader failed reading zip archive: failed finding central directory

In [ ]:
embeddings

In [ ]:

with open(self.prot_embedding_path.joinpath("prot_to_idx.json"), "r") as f:
    id2_idx = json.load(f)

# logger.info(id2_idx)
logger.info(embeddings.shape)
prot_with_emb = []
prot_wo_emb = []

for prot in proteins:
    if prot in id2_idx:
        prot_with_emb.append(id2_idx.get(prot))

    else:
        prot_wo_emb.append(prot)
logger.info("--- Prots with embeddings ---")
logger.info(prot_with_emb)

logger.info("--- Prots without embeddings ---")
logger.info(prot_wo_emb)

logger.info(
    f"--- prots wo: {len(prot_wo_emb)} prots with {len(prot_with_emb)} ---")
logger.info(f"--- prots in index: {len(id2_idx)} ----")
logger.info(
    f"--- proteins checked in index: {len([x for x in proteins if x in id2_idx])} ----")
logger.info(embeddings.shape)
prot_indices = torch.tensor(prot_with_emb)

prot_embeddings = embeddings[prot_indices]

logger.info("--- protein embeddings ---")
logger.info(prot_embeddings.shape)
return prot_embeddings